This notebook helps test a local or pip installed copy of llmsherpa with the ingestor core code

In [1]:
!pip install llmsherpa

You should consider upgrading via the 'C:\Users\fabia\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


## Add llmsherpa
###  This is used to parse PDFs and allows for smart chunking based on the documents structure (chapters, sections, tables, paragraphs).

In [1]:
import os, sys
%load_ext autoreload
from llmsherpa.readers import LayoutPDFReader
from IPython.core.display import display, HTML
%autoreload 2

C:\Users\fabia\AppData\Local\Temp\ipykernel_5804\1052947477.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
directory_path = "/Users/fabia/Desktop/testapi"
sys.path.insert(0, directory_path)


llmsherpa_api_url = "http://localhost:5001/api/parseDocument?renderFormat=all&useNewIndentParser=true"
pdf_url = "data/et200sp_system_manual_en-US_en-US_stripped.pdf"
# pdf_url = "data/test_manual_new.pdf"


do_ocr = True
if do_ocr:
    llmsherpa_api_url = llmsherpa_api_url + "&applyOcr=yes"
pdf_reader = LayoutPDFReader(llmsherpa_api_url)
doc = pdf_reader.read_pdf(pdf_url)

## Complete parsed document:

In [4]:
# HTML(doc.sections()[0].to_html(include_children=True, recurse=True))
# doc.sections()[1].block_json
# doc.sections()[0].to_text()
# doc.sections()[1].bbox
# llmsherpa.readers.Layout
HTML(doc.to_html())

①,BusAdapter
②,Mounting rail
③,Reference identification label
④,CPU/interface module
⑤,Light-colored BaseUnit BUD with infeed of supply voltage
⑥,Dark-colored BaseUnits BUB for conducting the potential group further
⑦,BaseUnit for motor starters
⑧,Potential distributor module
⑨,Ex BaseUnit for Ex power module
⑩,Ex BaseUnit for Ex I/O module
⑪,Server module (included in the scope of supply of the CPU/interface module)


## Parsed Chunks

In [5]:
for chunk in doc.chunks():
    print(chunk.to_context_text())
    print("-"*80)

SYSTEM MANUAL
11/2023Edition
--------------------------------------------------------------------------------
SIMATIC > ET 200SP > System overview > 5.1 What is the SIMATIC ET 200SP distributed I/O system? > SIMATIC ET 200SP
SIMATIC ET 200SP is a scalable and highly flexible distributed I/O system for connecting process signals to a higher-level controller via a fieldbus.
--------------------------------------------------------------------------------
SIMATIC > ET 200SP > System overview > 5.1 What is the SIMATIC ET 200SP distributed I/O system? > SIMATIC ET 200SP > Area of application
Thanks to its multifunctionality, the SIMATIC ET 200SP distributed I/O system is suitable for a wide range of applications.
Its scalable design allows you to tailor your configuration to local requirements.
Various CPUs/interface modules are available for connection to PROFINET IO, PROFIBUS DP, EtherNet/IP or Modbus TCP.
--------------------------------------------------------------------------------
SIM

In [7]:
for chunk in doc.chunks():
    if chunk.parent.title == "Mounting rules for reducing the thermal load":
        print(chunk.to_context_text())
        print()
        print("-"*80)


7.1 Basics > Mounting rules for reducing the thermal load
The following rules reduce the thermal load of the ET 200SP distributed I/O system in the control cabinet:
• Separate 2 modules with high power dissipation with a module of low power dissipation or by an empty space.
• Mix modules with higher power dissipation and modules with less power dissipation.
For example, modules with 16 outputs have a higher power dissipation than modules with 8 outputs.
• You should give preference to the horizontal mounting position.
• For vertical mounting position, plug modules with high power dissipation at the top, the interface module/CPU at the bottom.
7.2 Installation conditions for motor starters

--------------------------------------------------------------------------------
7.1 Basics > Mounting rules for reducing the thermal load
Mount an ET 200SP station with modules with high power dissipation in the lower area of the control cabinet.

----------------------------------------------------

### Post process the parsed chunks
As the parsed chunks can be to graniular we merge chunks that belong to the same (sub)section. We also merge the corresponding metadata like section, page_idx, parent, ...

In [8]:
def get_chunk_parent_titles(chunk):
    # Build a string that consists of recursively getting the chunk parent titles
    parent_titles = ""
    parent = chunk.parent
    while parent:
        try:
            parent_titles = parent.title + " > " + parent_titles
            parent = parent.parent
        except Exception as e:
            break
    # Remove the trailing " >"
    parent_titles = parent_titles.rstrip(" >")
    return parent_titles

In [11]:
# We want the full context_text without the section title. The section title is included in the metadata

def remove_first_line(text):
    lines = text.splitlines(True)[1:]  # splitlines(True) keeps the line breaks
    return ''.join(lines)

beforeString = doc.chunks()[12].to_context_text()
print("Before: ")
print(beforeString)
print()
print("After: ")
afterString = remove_first_line(beforeString)
print(afterString)

Before: 
SIMATIC > ET 200SP > System overview > 5.2 What are fail-safe automation systems and fail-safe modules? > Safety Integrated
Safety Integrated is the integrated safety concept for automation and drive technology from Siemens.

After: 
Safety Integrated is the integrated safety concept for automation and drive technology from Siemens.


### Parse new custom chunks
We merge chunks and parse their metadata into new custom chunks

In [117]:
# Dictionary to store merged content
merged_chunks = {}

# Iterate through each chunk and merge content
for chunk in doc.chunks():
    parent_section_hierarchy = get_chunk_parent_titles(chunk) # E.g. 'Installation > 7.3 Mounting the CPU/interface module > Dismantling the CPU/interface module'
    parent_section = chunk.parent.title # E.g. 'Dismantling the CPU/interface module'
    text = remove_first_line(chunk.to_context_text())
    page_idx = chunk.page_idx + 1 
    level = chunk.level

    if parent_section_hierarchy not in merged_chunks:
        merged_chunks[parent_section_hierarchy] = {
            "text": text,
            "start_page": page_idx,
            "end_page": page_idx,
            "parent_section": parent_section,
            "level": level
        }
    else:
        merged_chunks[parent_section_hierarchy]["text"] += "\n" + text
        merged_chunks[parent_section_hierarchy]["end_page"] = max(merged_chunks[parent_section_hierarchy]["end_page"], page_idx)

# Convert the merged dictionary back to list of chunks with updated page_nr
final_chunks = []
for parent_section_hierarchy, data in merged_chunks.items():
    start_page = data["start_page"]
    end_page = data["end_page"]
    if start_page == end_page:
        page_nr = str(start_page) # If there is only one page set that as the page number
    else:
        page_nr = f"{start_page}-{end_page}" # If there are multiple pages set the range as the page number
    
    final_chunks.append({
        "parent_section_hierarchy": parent_section_hierarchy,
        "text": data["text"],
        "page_nr": page_nr,
        "parent_section": data["parent_section"],
        "level": data["level"]
    })

# Output the final chunks
for chunk in final_chunks:
    print(f"Parent Section Hierarchy: {chunk['parent_section_hierarchy']}")
    print(f"Page Nr: {chunk['page_nr']}")
    print(f"Parent Section: {chunk['parent_section']}")
    print(f"Level: {chunk['level']}")
    print(f"Text: {chunk['text']}")
    print("="*80)


Parent Section Hierarchy: SYSTEM MANUAL
Page Nr: 1
Parent Section: SYSTEM MANUAL
Level: 1
Text: 11/2023Edition
Parent Section Hierarchy: SIMATIC > ET 200SP > System overview > 5.1 What is the SIMATIC ET 200SP distributed I/O system? > SIMATIC ET 200SP
Page Nr: 2
Parent Section: SIMATIC ET 200SP
Level: 5
Text: SIMATIC ET 200SP is a scalable and highly flexible distributed I/O system for connecting process signals to a higher-level controller via a fieldbus.
Parent Section Hierarchy: SIMATIC > ET 200SP > System overview > 5.1 What is the SIMATIC ET 200SP distributed I/O system? > SIMATIC ET 200SP > Area of application
Page Nr: 4
Parent Section: Area of application
Level: 6
Text: Thanks to its multifunctionality, the SIMATIC ET 200SP distributed I/O system is suitable for a wide range of applications.
Its scalable design allows you to tailor your configuration to local requirements.
Various CPUs/interface modules are available for connection to PROFINET IO, PROFIBUS DP, EtherNet/IP or Mod

### Print chunk metadata

In [118]:
def print_chunk_metadata(chunk):
    # print("Full json: ", chunk.block_json)
    # print()
    print("Page Nr: ", chunk.page_idx + 1)
    print("Tag: ", chunk.tag)
    print("Parent Section: ", chunk.parent.title)
    print("Level in the hierarchy: ", chunk.level)
    print("Parent Section Hierarchy: ", get_chunk_parent_titles(chunk))
    # print("Text: ", chunk.to_text())
    print()
    print("Context Text: ", chunk.to_context_text())
    print()
    print("="*100)


chunk = doc.chunks()[26]
print_chunk_metadata(chunk)


Page Nr:  8
Tag:  table
Parent Section:  Example of a configuration with fail-safe I/O modules
Level in the hierarchy:  5
Parent Section Hierarchy:  SIMATIC > ET 200SP > System overview > 5.3 How are SIMATIC Safety F-systems structured with ET 200SP? > Example of a configuration with fail-safe I/O modules

Context Text:  SIMATIC > ET 200SP > System overview > 5.3 How are SIMATIC Safety F-systems structured with ET 200SP? > Example of a configuration with fail-safe I/O modules
 | ① | Interface module
 | ② | Light-colored BaseUnit BUD with infeed of supply voltage
 | ③ | Dark-colored BaseUnits BUB for conducting the potential group further
 | ④ | I/O module
 | ⑤ | Server module (ships with the interface module)
 | ⑥ | Fail-safe I/O modules
 | ⑦ | BusAdapter
 | ⑧ | Mounting rail
 | ⑨ | Reference identification label




In [125]:
def print_custom_chunk(chunk):
    print("Page Nr: ", chunk['page_nr'])
    print("Parent Section: ", chunk['parent_section'])
    print("Parent Section Hierarchy: ", chunk['parent_section_hierarchy'])
    print("Level in the hierarchy: ", chunk['level'])
    print()
    print("Text:", chunk['text'])
    print()
    print("-"*100)

chunk = final_chunks[10]
print_custom_chunk(chunk)


Page Nr:  6-7
Parent Section:  Area of application of ET 200SP with fail-safe I/O modules
Parent Section Hierarchy:  SIMATIC > ET 200SP > System overview > 5.2 What are fail-safe automation systems and fail-safe modules? > Area of application of ET 200SP with fail-safe I/O modules
Level in the hierarchy:  5

Text: By using the ET 200SP distributed I/O system with fail-safe I/O modules, you are replacing conventional safety engineering configurations.
This includes the replacement of switching devices for emergency STOP, protective door monitors, two-hand operation, etc.
5.3 How are SIMATIC Safety F-systems structured with ET 200SP?

----------------------------------------------------------------------------------------------------


## Parsed Tables

In [17]:
for table in doc.tables():
    print(table.to_text())

 | ① | BusAdapter
 | ② | Mounting rail
 | ③ | Reference identification label
 | ④ | CPU/interface module
 | ⑤ | Light-colored BaseUnit BUD with infeed of supply voltage
 | ⑥ | Dark-colored BaseUnits BUB for conducting the potential group further
 | ⑦ | BaseUnit for motor starters
 | ⑧ | Potential distributor module
 | ⑨ | Ex BaseUnit for Ex power module
 | ⑩ | Ex BaseUnit for Ex I/O module
 | ⑪ | Server module (included in the scope of supply of the CPU/interface module)
 | ⑫ | Ex I/O module
 | ⑬ | Ex power module
 | ⑭ | ET 200SP motor starter
 | ⑮ | I/O module

 | ① | Interface module
 | ② | Light-colored BaseUnit BUD with infeed of supply voltage
 | ③ | Dark-colored BaseUnits BUB for conducting the potential group further
 | ④ | I/O module
 | ⑤ | Server module (ships with the interface module)
 | ⑥ | Fail-safe I/O modules
 | ⑦ | BusAdapter
 | ⑧ | Mounting rail
 | ⑨ | Reference identification label

 | SIL2 | Category 3 | (PL) Performance Level d
 | --- | --- | ---
 | SIL3 | Category 

## Parsed Sections

In [7]:
for section in doc.sections():
    print(section.to_text())

Installation
7.1 Basics
Introduction
Installation location
Mounting position
Mounting rail
NOTE
NOTE
NOTE
Minimum clearances
NOTE Ex module group
General rules for installation
WARNING Hazardous Voltage Can Cause Death, Serious Injury, or Property Damage.
NOTE
WARNING Protection from conductive contamination
Mounting rules for reducing the thermal load
7.2 Installation conditions for motor starters
Mechanical brackets
Designing interference-free motor starters
Mount the dummy module
NOTICE Ensure interference immunity
7.3 Mounting the CPU/interface module
Introduction
Requirement
Required tools
Mounting the CPU/interface module
Dismantling the CPU/interface module
NOTE
7.4 Installing ET 200SP R1
Introduction
Requirement
Tools required
Mounting the ET 200SP R1 system


## Search for specific sections/tables in the document:
### Here Section: 5.2 What are fail-safe automation systems and fail-safe modules?

In [277]:
def get_section_text(doc, section_title):
    """
    Extracts the text from a specific section in a parsed PDF document.

    Parameters:
    - doc (Document): A Document object from the llmsherpa.readers.layout_reader library.
    - section_title (str): The title of the section to extract.

    Returns:
    - str: The HTML representation of the section's content, or a message if the section is not found.
    """

    selected_section = None

    # Find the desired section by title
    for section in doc.sections():
        if section.title == section_title:
            selected_section = section
            break

    # If the section is not found, return a message
    if not selected_section:
        return f"No section titled '{section_title}' found."

    # Return the full content of the section as HTML
    return selected_section.to_html(include_children=True, recurse=True)

In [278]:
section_text = get_section_text(doc, 'Mounting the CPU/interface module')
HTML(section_text)

## Create RAG

### Create ChromaDB
ChromaDB stores the vectorstore (embeddings) persistent on the disc to be re-used

In [126]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("MergedCollection")

### Setup LLM model and embeddings model

In [127]:
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

llm = Ollama(model="llama3", request_timeout=120.0)
Settings.llm = llm

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.embed_model = embed_model

In [128]:
from llama_index.core import VectorStoreIndex
from llama_index.core.storage.storage_context import StorageContext
from llama_index.core import Document

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


### Insert parsed document chunks into the vectorstore

### Insert chunk with metadata into the vectorstore index

In [84]:
# from llama_index.core.schema import MetadataMode

# # If there is no existing vector store create a new one and assign it to the index
# index = VectorStoreIndex([], storage_context=storage_context)

# documents = []
# for chunk_id, chunk in enumerate(doc.chunks()):
#     document = Document(text=chunk.to_context_text(), 
#                         id_=chunk_id,
#                         metadata={
#                               #"bock_idx": chunk.block_idx, # Not sure if needed
#                               "page_number": chunk.page_idx + 1, # We add 1 to the page index to match the actual page number
#                               # "tag": chunk.tag,
#                               "parent_section": chunk.parent.title,
#                               "parent_section_hierarchy": get_chunk_parent_titles(chunk)
#                               #,"hierarchy_level": chunk.level # Not sure if needed 
#                               },
#                         text_template="Metadata\n{metadata_str}\nContent:\n{content}",)
#     documents.append(document)
#     print_chunk_metadata(chunk)
                        
#     index.insert(document)

#     # # Print the context of the chunk for debugging
#     # if chunk_id < 10:
#     #     print("---------------------The LLM sees this:---------------------",)
#     #     print(document.get_content(metadata_mode=MetadataMode.LLM),)
#     #     print("---------------------The Embedding model sees this:---------------------",)
#     #     print(document.get_content(metadata_mode=MetadataMode.EMBED),)
#     #     print("="*100)


Page Nr:  1
Tag:  para
Parent Section:  SYSTEM MANUAL
Parent Section Hierarchy:  SYSTEM MANUAL
Level in the hierarchy:  1

Context Text:  SYSTEM MANUAL
11/2023Edition

Page Nr:  2
Tag:  para
Parent Section:  SIMATIC ET 200SP
Parent Section Hierarchy:  SIMATIC > ET 200SP > System overview > 5.1 What is the SIMATIC ET 200SP distributed I/O system? > SIMATIC ET 200SP
Level in the hierarchy:  5

Context Text:  SIMATIC > ET 200SP > System overview > 5.1 What is the SIMATIC ET 200SP distributed I/O system? > SIMATIC ET 200SP
SIMATIC ET 200SP is a scalable and highly flexible distributed I/O system for connecting process signals to a higher-level controller via a fieldbus.

Page Nr:  4
Tag:  para
Parent Section:  Area of application
Parent Section Hierarchy:  SIMATIC > ET 200SP > System overview > 5.1 What is the SIMATIC ET 200SP distributed I/O system? > SIMATIC ET 200SP > Area of application
Level in the hierarchy:  6

Context Text:  SIMATIC > ET 200SP > System overview > 5.1 What is the SI

### Insert custom merged chunks

In [130]:
from llama_index.core.schema import MetadataMode

# If there is no existing vector store create a new one and assign it to the index
index = VectorStoreIndex([], storage_context=storage_context)

documents = []
for chunk_id, chunk in enumerate(final_chunks):
    document = Document(text=chunk['text'], 
                        id_=chunk_id,
                        metadata={
                              #"bock_idx": chunk.block_idx, # Not sure if needed
                              "page_number": chunk['page_nr'], # We add 1 to the page index to match the actual page number
                              # "tag": chunk.tag,
                              "parent_section": chunk['parent_section'],
                              "parent_section_hierarchy": chunk['parent_section_hierarchy']
                              #,"hierarchy_level": chunk['level'] # Not sure if needed 
                              },
                        text_template="Metadata\n{metadata_str}\nContent:\n{content}",)
    documents.append(document)
                        
    index.insert(document)

    # # Print the context of the chunk for debugging
    # if chunk_id < 10:
    #     print("---------------------The LLM sees this:---------------------",)
    #     print(document.get_content(metadata_mode=MetadataMode.LLM),)
    #     print("---------------------The Embedding model sees this:---------------------",)
    #     print(document.get_content(metadata_mode=MetadataMode.EMBED),)
    #     print("="*100)


Load existing vectorstore if one exists

In [15]:
# load your index from stored vectors
index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)

In [131]:
documents

[Document(id_='0', embedding=None, metadata={'page_number': '1', 'parent_section': 'SYSTEM MANUAL', 'parent_section_hierarchy': 'SYSTEM MANUAL'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='11/2023Edition', start_char_idx=None, end_char_idx=None, text_template='Metadata\n{metadata_str}\nContent:\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='1', embedding=None, metadata={'page_number': '2', 'parent_section': 'SIMATIC ET 200SP', 'parent_section_hierarchy': 'SIMATIC > ET 200SP > System overview > 5.1 What is the SIMATIC ET 200SP distributed I/O system? > SIMATIC ET 200SP'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='SIMATIC ET 200SP is a scalable and highly flexible distributed I/O system for connecting process signals to a higher-level controller via a fieldbus.', start_char_idx=None, end_char_idx=None, text_template='Metadata\n{metadata_str}\nContent:\n{co

In [133]:
def print_all_chunks():
    for chunk in final_chunks:
        print_custom_chunk(chunk)

print_all_chunks()

Page Nr:  1
Parent Section:  SYSTEM MANUAL
Parent Section Hierarchy:  SYSTEM MANUAL
Level in the hierarchy:  1

Text: 11/2023Edition

----------------------------------------------------------------------------------------------------
Page Nr:  2
Parent Section:  SIMATIC ET 200SP
Parent Section Hierarchy:  SIMATIC > ET 200SP > System overview > 5.1 What is the SIMATIC ET 200SP distributed I/O system? > SIMATIC ET 200SP
Level in the hierarchy:  5

Text: SIMATIC ET 200SP is a scalable and highly flexible distributed I/O system for connecting process signals to a higher-level controller via a fieldbus.

----------------------------------------------------------------------------------------------------
Page Nr:  4
Parent Section:  Area of application
Parent Section Hierarchy:  SIMATIC > ET 200SP > System overview > 5.1 What is the SIMATIC ET 200SP distributed I/O system? > SIMATIC ET 200SP > Area of application
Level in the hierarchy:  6

Text: Thanks to its multifunctionality, the SIMATI

### Query llm with context using Ollama Serve

Create prompt template based on LLAMA3 format

In [67]:
from llama_index.core import PromptTemplate

templateQA = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an assistant for answering questions about a technical manual. Use the following pieces of retrieved context from the manual to answer the question. \
If you don't know the answer, just say that you don't know. Be as detailed as possible.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

Question: {query_str}
Context: {context_str}
Answer: <|eot_id|>
<|start_header_id|>assistant<|end_header_id|>"""

# templateQA_instruction_generation = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

# You are a helper in creating assembly instructions based on a technical manual. Use the following pieces of retrieved context from the manual and the question to produce instructions. \
# If you don't know the answer, just say that you don't know. Be as detailed as possible.<|eot_id|>
# <|start_header_id|>user<|end_header_id|>

# Question: {query_str}
# Context: {context_str}
# Answer: <|eot_id|>
# <|start_header_id|>assistant<|end_header_id|>"""

# templateQA = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

# You are an assistant for creating assembly instructions for electrical components based on a technical manual. Only use the retrieved context from the manual and the question to produce instructions. \
# Always include an Introduction, the Requirement, the Required Tools, the steps for mounting/installing the component and the parent_section.<|eot_id|>
# <|start_header_id|>user<|end_header_id|>

# Question: {query_str}
# Context: {context_str}
# Answer: <|eot_id|>
# <|start_header_id|>assistant<|end_header_id|>"""

templateSummary = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Context information from multiple sources is below. Given the information from multiple sources and not prior knowledge, answer the query.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

Question: {query_str}
Context: {context_str}
Answer: <|eot_id|>
<|start_header_id|>assistant<|end_header_id|>"""

templateRefine = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Based on the original question and an existing answer, refine the existing answer (only if needed) with the additional context below.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

Original Question: {query_str}
Existing Answer: {existing_answer}
Context: {context_str}
Answer: <|eot_id|>
<|start_header_id|>assistant<|end_header_id|>"""

qa_template = PromptTemplate(templateQA)
summary_template = PromptTemplate(templateSummary)
refine_template = PromptTemplate(templateRefine)

# you can create text prompt (for completion API)
prompt = refine_template.format(context_str=..., query_str=..., existing_answer=...)
print("Prompt:")
print(prompt)

print()
print("="*100)
print()

# or easily convert to message prompts (for chat API)
messages = refine_template.format_messages(context_str=..., query_str=..., existing_answer=...)
print("Messages:")
print(messages)

Prompt:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Based on the original question and an existing answer, refine the existing answer (only if needed) with the additional context below.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

Original Question: Ellipsis
Existing Answer: Ellipsis
Context: Ellipsis
Answer: <|eot_id|>
<|start_header_id|>assistant<|end_header_id|>


Messages:
[ChatMessage(role=<MessageRole.USER: 'user'>, content='<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nBased on the original question and an existing answer, refine the existing answer (only if needed) with the additional context below.<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n\nOriginal Question: Ellipsis\nExisting Answer: Ellipsis\nContext: Ellipsis\nAnswer: <|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>', additional_kwargs={})]


### Helper function for printing context nodes
This can be used to find out which context was retrieved and used to create the response

In [39]:
def print_context(response):
    print("Nr of context nodes: ", len(response.source_nodes))
    print()
    print("="*150)
    for node in response.source_nodes:
        print("---------------------Metadata:---------------------")
        print(node.node.metadata)
        print("---------------------Text:---------------------")
        print(node.text)
        print("="*150)

## Customize Retrievers
### Based on Similarity
Uses top-k = 10 and similarity cutoff = 60% 

### TODO: Add own retrieval algorithm based on sections (Install, Mounting, Assembly, Disassembly, ...)
https://docs.llamaindex.ai/en/stable/understanding/querying/querying/

https://docs.llamaindex.ai/en/stable/module_guides/querying/node_postprocessors/

In [63]:
#A dummy node-postprocessor can be implemented in just a few lines of code:
from typing import List, Optional

from llama_index.core import QueryBundle
from llama_index.core.postprocessor.types import BaseNodePostprocessor
from llama_index.core.schema import NodeWithScore


class DummyNodePostprocessor(BaseNodePostprocessor):
    def _postprocess_nodes(
        self, nodes: List[NodeWithScore], query_bundle: Optional[QueryBundle]
    ) -> List[NodeWithScore]:
        # subtracts 1 from the score
        for n in nodes:
            # n.score -= 1
            parent = n.metadata["parent_section"]
            if any(substring in parent for substring in ["Introduction", "Requirements", "Installing"]):
                print("Substring found")
                n.score = 1
            else:
                print("Substring not found") 
                n.score = 0

        return nodes

### Based on Keywords
Include and exclude required keywords from the context

In [13]:
# Helper function to view the prompt format
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k} -> " f"**Text:**"
        print(text_md)
        print(p.get_template())

TODO: Explore different retrievers: https://docs.llamaindex.ai/en/stable/module_guides/indexing/index_guide/

### Metadata filters
This can't be applied because we can only search for exact keywords in the metadata, but the section titles vary too much.

**We could theoretically apply that to queries concerning only the "Introduction", "Requirement(s)" and "Tools required" as these keywords are always the same.**

In [110]:
# from llama_index.core.vector_stores.types import MetadataFilters, ExactMatchFilter, MetadataFilter

# included_section_keywords = ["Introduction", "Requirement", "Required tools", "Installing", "Preparing", "Mounting", "Attaching", "Dismantling"
#                              "Removing", "Disassembling", "Uninstalling", "Tools required", "Required accessories", "Connecting", "Plugging", 
#                              "Releasing", "Removing"]

# filters = filters=[
#         MetadataFilter(key='parent_section', value=section, operator='==') 
#         for section in included_section_keywords
#     ]

# metadataFilters = MetadataFilters(filters=filters, condition='or')

In [111]:
from llama_index.core import get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor,KeywordNodePostprocessor, MetadataReplacementPostProcessor, SentenceTransformerRerank

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
    # filters=metadataFilters
)


# configure response synthesizer: https://docs.llamaindex.ai/en/stable/module_guides/querying/response_synthesizers/
response_synthesizer = get_response_synthesizer(response_mode="compact") # response mode "refine" for more detailed responses

# This works with or logic, so if any of the required keywords are found, the node is kept
node_postprocessors = [
    # KeywordNodePostprocessor(
    #     required_keywords=["Basic component"]#, exclude_keywords=["Italy"]
    #     # required_keywords=[]
    # ),
    # SimilarityPostprocessor(similarity_cutoff=0.6),
    # MetadataReplacementPostProcessor(target_metadata_key="window")
    # DummyNodePostprocessor()
    # SentenceTransformerRerank(model="cross-encoder/ms-marco-MiniLM-L-2-v2", top_n=10)
]

#"default": "create and refine" an answer by sequentially going through each retrieved Node; This makes a separate LLM call per Node. Good for more detailed answers.
#"tree_summarize": Given a set of Node objects and the query, recursively construct a tree and return the root node as the response. Good for summarization purposes.
response_mode = "tree_summarize" 

query_engine = RetrieverQueryEngine.from_args(
    retriever=retriever, 
    response_synthesizer=response_synthesizer, 
    node_postprocessors=node_postprocessors, 
    # response_mode=response_mode
)

# Update RetrieverQueryEngine prompt template to match LLAMA3 format
query_engine.update_prompts({"response_synthesizer:text_qa_template": qa_template,
                            "response_synthesizer:summary_template": summary_template,
                            "response_synthesizer:refine_template": refine_template})

# prompts_dict = query_engine.get_prompts()
# display_prompt_dict(prompts_dict)

In [96]:
documents[5].metadata

{'page_number': '6',
 'parent_section': 'Figure 5-2 Configuration example of the ET 200SP',
 'parent_section_hierarchy': 'SIMATIC > ET 200SP > System overview > 5.1 What is the SIMATIC ET 200SP distributed I/O system? > SIMATIC ET 200SP > Configuration example > Figure 5-2 Configuration example of the ET 200SP'}

In [114]:
# query
response = query_engine.query("How do I mount/install a base unit?")
print(response)

print()
print("Provided context: ")
print_context(response)

According to the provided context, the steps to mount/install a base unit are:

1. Hook the BaseUnit into the DIN rail from above.
2. Swing the BaseUnit to the rear until the BaseUnit audibly engages.
3. Slide the individual BaseUnits to the left to the previous BaseUnit until they audibly engage.
Assemble the BaseUnits only on the DIN rail.

It is also recommended to watch a video sequence (https://support.automation.siemens.com/WW/view/en/95886218) for further guidance, as mentioned in the context.

Provided context: 
Nr of context nodes:  6

---------------------Metadata:---------------------
{'page_number': '66', 'parent_section': 'Mounting a BaseUnit', 'parent_section_hierarchy': 'Mounting a BaseUnit'}
---------------------Text:---------------------
Proceed as follows to mount a BaseUnit for motor starters:
1. Hook the BaseUnit into the DIN rail from above.
2. Swing the BaseUnit to the rear until the BaseUnit audibly engages.
3. Slide the individual BaseUnits to the left to the pr

In [115]:
# query
response = query_engine.query("I have two questions answer them separately. How do I mount a cpu module on the system rail and how do I dismantle a cpu module? In which section and on which page can I find this information?")
print(response)

print()
print("Actual context: ")
print_context(response)

Here are your answers:

**How do I mount a CPU module on the system rail?**

According to the manual, you can find this information in Section "Mounting the CPU/interface module" starting from page 7. To install a CPU/interface module, follow these steps:

1. Install the CPU/interface module on the mounting rail.
2. Swivel the CPU/interface module towards the back until you hear the mounting rail release button click into place.

You can also watch the video sequence mentioned in the manual for more detailed instructions: https://support.automation.siemens.com/WW/view/en/95886218

**How do I dismantle a CPU module?**

According to the manual, you can find this information starting from page 8. To remove the CPU/interface module, follow these steps:

1. Switch off the supply voltage for the CPU/interface module.
2. Press the mounting rail release button on the first BaseUnit.
At the same time, shift the CPU/interface module parallel to the left until it detaches from the rest of the mod

In [88]:
# query
response = query_engine.query("Tell me the tools I need for the installation of ET 200SP R1.")
print(response)

print()
print("Actual context: ")
print_context(response)

According to the provided context, the tools you need for the installation of ET 200SP R1 are:

* 3 to 3.5 mm screwdriver (only for mounting and removing the BusAdapter)

This information is found in the "Tools required" section on page 8 of the metadata.

Actual context: 
Nr of context nodes:  10

---------------------Metadat:---------------------
{'page_number': 8, 'tag': 'para', 'parent_section': 'Mounting the ET 200SP R1 system', 'parent_section_hierarchy': 'Installation > 7.4 Installing ET 200SP R1 > Mounting the ET 200SP R1 system'}
---------------------Text:---------------------
Installation > 7.4 Installing ET 200SP R1 > Mounting the ET 200SP R1 system
To mount the ET 200SP R1 system, proceed as follows:
1. Hang the BaseUnit BU type M0 onto the SIMATIC system rail.
2. Swivel the BaseUnit BU type M0 backwards until the system rail release audibly engages.
3. Plug the IM 155-6 PN R1 interface modules onto the BaseUnit BU type M0 until the lock audibly engages.
4. Plug the 24 V DC

### TODO: Explore structured output
https://docs.llamaindex.ai/en/stable/module_guides/querying/structured_outputs/query_engine/

Allows for parsing the llm output into an object like Manual entry: string section, string text

### TODO: Test llamaparse for PDF parsing
https://docs.llamaindex.ai/en/stable/module_guides/loading/connector/llama_parse/

--> DONE: Parsing and indexing is superior to llmsherpa out of the box but requires payed API key

### TODO: Benchmarking


- Retrieval Evaluation: This assesses the accuracy and relevance of the information retrieved by the system.
- Response Evaluation: This measures the quality and appropriateness of the responses generated by the system based on the retrieved information.


In [33]:
def add_qa_to_list(lst, question, answer):
    lst.append({question: answer})
    return lst

qa_list = []
add_qa_to_list(qa_list, "What is the first step in mounting the CPU/interface module?", "Install the CPU/interface module on the mounting rail.")
add_qa_to_list(qa_list, "What tools are needed for the installation of ET 200SP R1?", "3 to 3.5 mm screwdriver (only for mounting and removing the BusAdapter)")
add_qa_to_list(qa_list, "What are the requirements for installing a ET 200SP R1??", "The SIMATIC system rail is installed.")
add_qa_to_list(qa_list, "Which type of mounting rails can I use for motor starters?", "– 35x15 mm DIN rail in accordance with DIN EN 60715\n– 35x7.5 mm DIN rail in accordance with DIN EN 60715\n– SIMATIC S7 mounting rail")


[{'What is the first step in mounting the CPU/interface module?': 'Install the CPU/interface module on the mounting rail.'},
 {'What tools are needed for the installation of ET 200SP R1?': '3 to 3.5 mm screwdriver (only for mounting and removing the BusAdapter)'},
 {'What are the requirements for installing a ET 200SP R1??': 'The SIMATIC system rail is installed.'},
 {'Which type of mounting rails can I use for motor starters?': '– 35x15 mm DIN rail in accordance with DIN EN 60715\n– 35x7.5 mm DIN rail in accordance with DIN EN 60715\n– SIMATIC S7 mounting rail'}]

In [39]:
def get_context_string(response):
    string = ""
    for node in response.source_nodes:
        string += f"Context Metadata:\n"
        string += f"{node.node.metadata}\n"
        string += f"Context Text:\n"
        string += f"{node.text}\n"
        string += "\n\n"
    return string

In [53]:
templateEvalContextRelevance = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Your task is to evaluate if the provided context information is sufficient to answer a question correctly. \
You are provided with the context and the expected response. \n
You have two options to answer. Either YES/ NO. \n
Answer - YES, if all the information in the expected response are included in the context otherwise NO.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

Context: \n {context_str}\n
Expected Response: \n {expected_response}\n
Answer: <|eot_id|>
<|start_header_id|>assistant<|end_header_id|>"""

eval_template = PromptTemplate(templateEvalContextRelevance)

In [58]:
eval_index = VectorStoreIndex([])
eval_query_engine = eval_index.as_chat_engine()

def execute_evaluation(qa_entry):
    query = list(qa_entry.keys())[0]
    expected_repsonse = list(qa_entry.values())[0]

    # Execute query with question
    response = query_engine.query(query)

    # Get retrieved context
    context = get_context_string(response)
    expected_repsonse = expected_repsonse

    # Print everything for debugging
    print("--------------------------Query: --------------------------")
    print(query)
    print("--------------------------Response: --------------------------")
    print(response)
    print("--------------------------Context: --------------------------")
    print(context)
    print("--------------------------Expected Response: --------------------------")
    print(expected_repsonse)

    # Generate evaluation prompt
    eval_prompt = eval_template.format(context_str=context, expected_response=expected_repsonse)

    # Evaluate the context relevance
    eval_response = eval_query_engine.query(eval_prompt)

    print("--------------------------Evaluation : --------------------------")
    print(eval_response)

    return eval_response

In [59]:
execute_evaluation(qa_list[1])

--------------------------Query: --------------------------
What tools are needed for the installation of ET 200SP R1?
--------------------------Response: --------------------------
The typical tools and equipment needed for installing Siemens ET 200SP R1 compact controllers include screwdrivers, Allen wrenches, wire strippers, crimping tools, and a multimeter. Additionally, a programming device such as a laptop with the corresponding software may be required for configuration purposes.
--------------------------Context: --------------------------

--------------------------Expected Response: --------------------------
3 to 3.5 mm screwdriver (only for mounting and removing the BusAdapter)
--------------------------Evaluation : --------------------------
You would need a 3 to 3.5 mm screwdriver for mounting and removing the BusAdapter.


Response(response='You would need a 3 to 3.5 mm screwdriver for mounting and removing the BusAdapter.', source_nodes=[], metadata=None)